In [1]:
import pandas as pd

In [27]:
feat_import = pd.read_csv('data/output/feature_importance.csv')
feat_import

,imp,feature
0,0.012473,220045
1,0.006539,220046
2,0.005779,220047
3,0.009740,220179
4,0.008369,220180
...,...,...
1086,0.000758,ethnicity_6
1087,0.001348,ethnicity_7
1088,0.000268,insurance_0
1089,0.001640,insurance_1


In [24]:
d_labitems = pd.read_csv("mimiciv/1.0/icu/d_items.csv.gz")

In [49]:
feat_mask = feat_import.feature.apply(lambda x: x[0].isdigit())
feat_import_itemid = feat_import.feature[feat_mask].astype(int)
feat_import_itemid

0      220045
1      220046
2      220047
3      220179
4      220180
        ...  
365    227933
366    227602
367    227614
368    228628
369    228731
Name: feature, Length: 370, dtype: int64

In [59]:
metadata = d_labitems[d_labitems.itemid.isin(feat_import_itemid)][['itemid', 'label']]
metadata['itemid'] = metadata.itemid.astype('str')
metadata

,itemid,label
1,220045,Heart Rate
2,220046,Heart rate Alarm - High
3,220047,Heart Rate Alarm - Low
5,220050,Arterial Blood Pressure systolic
6,220051,Arterial Blood Pressure diastolic
...,...,...
3796,228184,SVV (PiCCO)
3797,228185,SVRI (PiCCO)
3841,228872,HM II- Mean BP
3842,228873,HM II- Flow


In [60]:
feat_import.merge(metadata, left_on='feature', right_on='itemid')

,imp,feature,itemid,label
0,0.012473,220045,220045,Heart Rate
1,0.006539,220046,220046,Heart rate Alarm - High
2,0.005779,220047,220047,Heart Rate Alarm - Low
3,0.009740,220179,220179,Non Invasive Blood Pressure systolic
4,0.008369,220180,220180,Non Invasive Blood Pressure diastolic
...,...,...,...,...
365,0.000000,227933,227933,Recovery O2 sat - Aerobic Activity Response
366,0.000000,227602,227602,Impaired Skin Length #9
367,0.000000,227614,227614,Impaired Skin Width #9
368,0.000000,228628,228628,Pressure Ulcer #9- Width


In [61]:
metadata.to_csv('data/output/metadata.csv')